Wikipedia analysis using Latent Dirichlet Allocation (LDA)
==========================================================

Download the Swedish Wikipedia dump

In [ ]:
# wget https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2

  

Extract the .xml-file

In [ ]:
# ls
# bzip2 -dk svwiki-20201120-pages-articles.xml.bz2

  

Copy file to DBFS cluster

In [ ]:

// val localpath="file:/databricks/driver/svwiki-20201120-pages-articles.xml"

// dbutils.fs.mkdirs("dbfs:/datasets/")

// dbutils.fs.cp(localpath, "dbfs:/datasets/")

// display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

In [ ]:
// Verify Dataset is in cluster
// display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

  

  

We now have a wikipedia dump ready to be processed

This page has some resources that can be used for Wikipedia parsing:
https://mindfulmachines.io/blog/2018/3/18/wikipedia-data-in-spark-and-scala-updated

In [ ]:
//---------------spark_import
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SQLContext

//----------------xml_loader_import
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{ LongWritable, Text }
import com.databricks.spark.xml._

import org.apache.hadoop.conf.Configuration

  

>     import org.apache.spark.SparkContext
>     import org.apache.spark.SparkConf
>     import org.apache.spark.sql.SQLContext
>     import org.apache.hadoop.io.LongWritable
>     import org.apache.hadoop.io.Text
>     import org.apache.hadoop.conf.Configuration
>     import org.apache.hadoop.io.{LongWritable, Text}
>     import com.databricks.spark.xml._
>     import org.apache.hadoop.conf.Configuration

In [ ]:

def readWikiDump(sc: SparkContext, file: String) : RDD[(Long, String)] = {
  val conf = new Configuration()
  conf.set(XmlInputFormat.START_TAG_KEY, "<page>")
  conf.set(XmlInputFormat.END_TAG_KEY, "</page>")
  val rdd = sc.newAPIHadoopFile(file, 
                                classOf[XmlInputFormat], 
                                classOf[LongWritable], 
                                classOf[Text], 
                                conf)
  rdd.map{case (k,v) => (k.get(), new String(v.copyBytes()))}
}

  

>     readWikiDump: (sc: org.apache.spark.SparkContext, file: String)org.apache.spark.rdd.RDD[(Long, String)]

In [ ]:
// Read the wiki dump

val dump = readWikiDump(sc, "dbfs:/datasets/svwiki-20201120-pages-articles.xml")

//dump.collect() This throws a memory error

dump.take(5)

  

>     dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[27] at map at command-738986526041391:11
>     res2: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>), (52187,<page>
>         <title>Afrika</title>
>         <ns>0</ns>
>         <id>2</id>
>         <revision>
>           <id>48459142</id>
>           <parentid>48374249</parentid>
>           <timestamp>2020-11-18T06:30:03Z</timestamp>
>           <contributor>
>             <username>LarskeBot</username>
>             <id>310602</id>
>           </contributor>
>           <minor />
>           <comment>/* Geografi */mellan X - Y → mellan X och Y med [[Project:AWB|AWB]]</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="37806" xml:space="preserve">{{templateStyles-tagg Faktamall}}
>     {| class=&quot;infobox&quot; style=&quot;font-size:90%;&quot; width=&quot;300&quot;
>     |+ &lt;big&gt;'''Afrika'''&lt;/big&gt;
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Africa (orthographic projection).svg|280px|Afrika i världen.]]
>     ----
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Afrika politisk.png|280px|Politisk karta.]]
>     ----
>     |-
>     | colspan=&quot;2&quot; |
>     |-
>     | '''Area:''' &lt;br /&gt; '''Folkmängd:''' &lt;br /&gt; '''Antal stater:''' &lt;br /&gt; '''Autonoma regioner:''' &lt;br /&gt; '''Högsta punkt:''' &lt;br /&gt; '''Lägsta punkt:''' || 30&amp;nbsp;221&amp;nbsp;532&amp;nbsp;km² &lt;br /&gt; 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;000&lt;ref name=&quot;esa.un.org&quot;/&gt; (2010) &lt;br /&gt; 54 &lt;br /&gt; 8 &lt;br /&gt; [[Kilimanjaro]]&amp;nbsp;5&amp;nbsp;895&amp;nbsp;[[m ö.h.|m&amp;nbsp;ö.h.]] &lt;br /&gt; [[Assalsjön]]&amp;nbsp;156&amp;nbsp;[[Meter över havet|m&amp;nbsp;u.h.]]
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     |-
>     | colspan=&quot;2&quot; | '''Stater'''
>     |-
>     | {{flaggbild2|Algeriet}} &lt;br /&gt; {{flaggbild2|Angola}} &lt;br /&gt; {{flaggbild2|Benin}} &lt;br /&gt; {{flaggbild2|Botswana}} &lt;br /&gt; {{flaggbild2|Burkina Faso}} &lt;br /&gt; {{flaggbild2|Burundi}} &lt;br /&gt; {{flaggbild2|Centralafrikanska republiken}} &lt;br /&gt; {{flaggbild2|Djibouti}} &lt;br /&gt; {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Ekvatorialguinea}} &lt;br /&gt; {{flaggbild2|Elfenbenskusten}} &lt;br /&gt; {{flaggbild2|Eritrea}} &lt;br /&gt; {{flaggbild2|Etiopien}} &lt;br /&gt; {{flaggbild2|Gabon}} &lt;br /&gt; {{flaggbild2|Gambia}} &lt;br /&gt; {{flaggbild2|Ghana}} &lt;br /&gt; {{flaggbild2|Guinea}} &lt;br /&gt; {{flaggbild2|Guinea-Bissau}} &lt;br /&gt; {{flaggbild2|Kamerun}} &lt;br /&gt; {{flaggbild2|Kap Verde}} &lt;br /&gt; {{flaggbild2|Kenya}} &lt;br /&gt; {{flaggbild2|Komorerna}} &lt;br /&gt; {{flaggbild2|Kongo-Brazzaville}} &lt;br /&gt; {{flaggbild2|Kongo-Kinshasa}} &lt;br /&gt; {{flaggbild2|Lesotho}} &lt;br /&gt; {{flaggbild2|Liberia}} &lt;br /&gt;{{flaggbild2|Libyen}} ||{{flaggbild2|Madagaskar}} &lt;br /&gt; {{flaggbild2|Malawi}} &lt;br /&gt; {{flaggbild2|Mali}} &lt;br /&gt; {{flaggbild2|Marocko}} &lt;br /&gt; {{flaggbild2|Mauretanien}} &lt;br /&gt; {{flaggbild2|Mauritius}} &lt;br /&gt; {{flaggbild2|Moçambique}} &lt;br /&gt; {{flaggbild2|Namibia}} &lt;br /&gt; {{flaggbild2|Niger}} &lt;br /&gt; {{flaggbild2|Nigeria}} &lt;br /&gt; {{flaggbild2|Rwanda}} &lt;br /&gt; {{flaggbild2|São Tomé och Príncipe}} &lt;br /&gt; {{flaggbild2|Senegal}} &lt;br /&gt; {{flaggbild2|Seychellerna}} &lt;br /&gt; {{flaggbild2|Sierra Leone}} &lt;br /&gt; {{flaggbild2|Somalia}} &lt;br /&gt; {{flaggbild2|Sudan}} &lt;br /&gt; {{flaggbild2|Sydafrika}} &lt;br /&gt; {{flaggbild2|Sydsudan}} &lt;br /&gt; {{flaggbild2|Swaziland}} &lt;br /&gt; {{flaggbild2|Tanzania}} &lt;br /&gt; {{flaggbild2|Tchad}} &lt;br /&gt; {{flaggbild2|Togo}} &lt;br /&gt; {{flaggbild2|Tunisien}} &lt;br /&gt; {{flaggbild2|Uganda}} &lt;br /&gt; {{flaggbild2|Zambia}} &lt;br /&gt; {{flaggbild2|Zimbabwe}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Stater som har delar av ytan i Afrika'''
>     |-
>     | {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Frankrike}} &lt;br /&gt; {{flaggbild2|Jemen}} &lt;br /&gt; {{flaggbild2|Portugal}} &lt;br /&gt; {{flaggbild2|Spanien}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Särskilda områden'''
>     &lt;br /&gt; {{flaggbild|Ceuta}} [[Ceuta]] ([[Spanien]])
>
>     {{flaggbild|Kanarieöarna}} [[Kanarieöarna]] ([[Spanien]])
>
>     {{flaggbild|Madeira}} [[Madeira]] ([[Portugal]])
>
>     {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>
>     {{flaggbild|Melilla}} [[Melilla]] ([[Spanien]])
>
>     {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>
>     {{flaggbild|Sankta Helena}} [[Sankta Helena]] ([[Storbritannien]])
>
>     [[Fil:Flag of Somaliland.svg|22px|Somaliland]] [[Somaliland]]
>
>     {{flaggbild2|Västsahara}} ([[Marocko]])
>     ----
>     '''Största städer (över 10 miljoner invånare med förorter):'''
>     &lt;br /&gt;{{flaggbild|Egypten}} [[Kairo]]
>     &lt;br /&gt;{{flaggbild|Kongo-Kinshasa}} [[Kinshasa]]
>     &lt;br /&gt;{{flaggbild|Nigeria}} [[Lagos]]
>     ----
>     [[Fil:Africa satellite.jpg|280px|Satellitfoto]]
>     |}
>     '''Afrika''' är [[jorden]]s näst största [[kontinent]] (efter [[Eurasien]]) och även jordens näst största världsdel efter [[Asien]], både vad gäller [[areal]] och [[folkmängd]]. Med världsdelens öar inräknade mäter Afrika 30 244 050&amp;nbsp;km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. Omkring 22 miljoner km² av dessa ligger i [[tropikerna]] vilket gör den afrikanska kontinenten till världens varmaste kontinent. I Afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. Dess längd i nordlig-sydlig riktning är omkring 8 000&amp;nbsp;km och dess största bredd omkring 7&amp;nbsp;800&amp;nbsp;km.
>
>     == Etymologi ==
>     Ordet &quot;Afrika&quot;s [[etymologi]]ska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika [[epok]]er.
>
>     I äldre tider kallades allt land väster om [[Egypten]] för &quot;[[Libyen]]&quot;. Enligt den [[Grekland|grek]]iska historieskrivaren [[Herodotos]] berättelse seglade [[fenicisk]]a sjömän på uppdrag av den egyptiska kungen Neko runt Libyen, det vill säga [[kontinent]]en Afrika redan omkring 600 f.Kr.
>
>     När [[Romerska riket|romarna]] först använde termen ''[[Africa terra]]'' avsåg de före [[andra puniska kriget]] endast trakten kring [[Karthago]], det vill säga motsvarande ungefär dagens Tunisien där själva staden Karthago låg samt stora delar av Libyen och Algeriet. Romarna uppfattade först [[Egypten]] som en del av [[Asien]], och det var först geografen [[Klaudios Ptolemaios|Ptolemaios]] som drog gränsen vid [[Suez]]. Under kejsartiden kom ''Africa'' att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. Namnet ska ha avsett &quot;landet där ''afri''&quot; (singularis: ''afer'') bodde. ''Afer'' avsåg namnet på ett folk som kan motsvara dagens [[berber]]. [[Viking]]arna under 800- och 900-talen kallade norra Afrika för &quot;Blåland&quot;.
>
>     == Geografi ==
>     {{Fördjupning|Afrikas geografi}}
>     Geologiskt är Afrika en mycket gammal kontinent, i huvudsak uppbyggd av [[urberg]]. Under jordens forntid var den en del av jättekontinenten [[Gondwana]]land. Denna gled isär under [[Jura (period)|juraperioden]] och [[Krita (geologi)|kritaperioden]] för 100-150 miljoner år sedan, och så fick Afrika sin nuvarande form.
>
>     Inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050&amp;nbsp;km² inklusive öarna. Afrika åtskiljs från [[Europa]] av [[Medelhavet]] och förenas med [[Asien]] i sitt nordöstra hörn genom [[näs]]et vid [[Suez]], omkring 130&amp;nbsp;km i bredd. Omkring 8 000&amp;nbsp;km skiljer den nordligaste punkten, [[Ras ben Sakka]] i [[Tunisien]], från dess sydligaste, [[Kap Agulhas]] i [[Sydafrika]]. Från [[Kap Verde]], dess västligaste punkt, till [[Ras Hafun]] i [[Somalia]] är det omkring 7 400&amp;nbsp;km. Afrikas kuster mäter omkring 28 000&amp;nbsp;km (1&amp;nbsp;km kust på 1 057&amp;nbsp;km²) varav [[Medelhavet]] står för 5 400&amp;nbsp;km, [[Atlanten]] 11 000&amp;nbsp;km, [[Indiska oceanen]] 8 700&amp;nbsp;km och [[Röda havet]] 2 900&amp;nbsp;km, och avsaknaden av djupare [[bukt]]er och betydande [[halvö]]ar blir uppenbar om man jämför med Europas 9 700 000&amp;nbsp;km² som omgärdas av 32 000&amp;nbsp;km kust (1&amp;nbsp;km kust på 278&amp;nbsp;km² land), eller för det [[fjord]]rika [[Norge]], där motsvarande siffra 385 000&amp;nbsp;km² land och en kuststräcka på 20 000&amp;nbsp;km (1&amp;nbsp;km kust per 19&amp;nbsp;km² land).
>
>     Afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. Jämfört med de andra kontinenterna utmärker sig Afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. Landområdena under 180 [[Meter över havet|m ö.h.]] upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i Asien, utgörs av isolerade berg och bergskedjor. Generellt ligger de högre [[platå]]erna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid [[Röda havet]]s mitt till [[Kongofloden]]s mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. Man kan således dela upp Afrika i fyra övergripande landområden:
>     # Slättlanden vid kusterna – ofta med [[Mangrove|mangroveträsk]] insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. Slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # [[Atlasbergen]] i norr som ligger isolerat av [[Sahara]]öknen som bitvis ligger under havsnivån.
>     # De södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # De nordliga och västra slättlanden, inklusive Sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     Skog täcker cirka en femtedel av Afrikas yta.
>
>     Hela Afrika har ett [[Tropiskt klimat|tropiskt]] eller [[Subtropiskt klimat]].
>
>     === Fauna ===
>
>     {{Fördjupning|Afrikas fauna}}
>
>     Afrika är huvudsakligen känt för den öppna [[savann]]en med väldiga hjordar av hovdjur som [[afrikansk buffel]], gnuer, [[antiloper]], [[giraff]]er och [[zebror]]. Dessa jagas av stora rovdjur som [[lejon]], [[hyenor]] och [[Gepard|geparder]]. Den afrikanska djungeln befolkas av [[ormar]] och [[primater]] samt av vattenlevande djur som [[krokodiler]] och [[groddjur]]. Afrika har även det största antalet arter som kan räknas till [[megafauna]]n, bland annat [[afrikanska elefanter]], [[trubbnoshörning]] och [[flodhäst]], på grund av att kontinenten bara i mindre utsträckning var påverkad av det [[massutdöende]] som skedde under [[pleistocen]].
>
>     == Historia ==
>     {{Fördjupning|Afrikas historia}}
>     Den första mänskliga migrationen, från kontinenten men även inom Afrikas kuster, kan spåras [[lingvistisk]]t, kulturellt och genom datoranalyser av [[genetik|genetiskt]] material. Någonstans i [[Östafrika]] tror dagens forskare att de första [[hominider]] uppkom. I [[Etiopien]] har man hittat ett två miljoner år gammalt skelett som uppkallades [[Australopithecus afarensis|Lucy]], den första kända mänskliga [[primater|primaten]].
>
>     === Tidigare civilisationer och handel ===
>     [[Fil:African-civilizations-map-pre-colonial.svg|miniatyr|höger|Några av de mest kända afrikanska civilisationerna. Civilisationer från olika tidsepoker visas parallellt.]]Lingvistiska spår tyder på att [[Bantu]]folk emigrerade från [[Västafrika]] in i [[khoisan]]folkets (de som tidigare kallades [[bushmän]] och [[hottentott]]er) område och undanträngde dem. Bantufolken använde en uppsättning grödor, bland annat [[kassava]] och [[jams]], som var mer lämpade för jordbruk i det [[tropik|tropiska]] Afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. Bantufolkens traditionella bosättningsområde sträcker sig från [[Sahara]]öknen till de [[Tempererat klimat|tempererade regionerna]] i söder. Historiskt har deras vapen varit spjut, sköld, pil och båge. Det för [[khoisanspråk]]ens karaktäristiska klickljuden återfinns i [[nguni (språk)|nguni]]- och [[bantu]]språk som till exempel [[xhosa]] och [[zulu]]. Khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra Afrika i länderna kring [[Kalahariöknen]].
>
>     Afrika söder om Sahara har bestått olika stater och riken, mindre stadsstater samt [[stam (samhällsvetenskap)|stammar]] och [[klan]]er. Det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska [[civilisation]]erna. Det fanns även statsbildningar i [[Nubien]], [[Ghana]], [[Timbuktu]], [[Kanem-Bornu]] och i nuvarande [[Zimbabwe]].
>
>     [[Nordafrika]] kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds [[Islam]] och den arabiska kulturen genom Sahara och vidare söderut. Arabiska och persiska handelsmän spred likaså Islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. Detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till Java, därmed den etniska, kulturella och språkliga mångfalden på Östafrikas kust, [[Madagaskar]] och [[Komorerna]].
>
>     === Arabvärlden och Europa utforskar kontinenten ===
>     Det arabiska intresset för Afrika går långt tillbaka, men handel med slavar från centra som Lamu och Zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.Kr. De miljoner slavar som exporterades till arabvärlden och österut under den [[arabiska slavhandeln]] finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. Slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel Sudan.&lt;ref&gt;[http://www.hrw.org/backgrounder/africa/sudanupdate.htm Human Rights Watch] [http://news.bbc.co.uk/2/hi/programmes/this_world/6458377.stm BBC World]&lt;/ref&gt;
>
>     Under [[1300-talet]] började vissa europeiska handelsexpeditioner få kontakt med Nord- och Västafrika.
>
>     Från 1500-talets mitt fram till förbudet mot slavhandel i Storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst Väst- och Centralafrika. Flera miljoner afrikaner såldes på detta vis som slavar till främst de Europeiska kolonierna i Amerika.
>
>     Tidiga kolonisatörer var [[araber]], [[Portugal|portugiser]] och [[Spanien|spanjorer]]. Därefter koloniserades kontinenten av framförallt av [[Storbritannien]], [[Frankrike]], [[Spanien]], [[Italien]], [[Belgien]], [[Nederländerna]], [[Tyskland]] och [[Portugal]].
>
>     === Koloniseringen ===
>     {{Fördjupning|Kapplöpningen om Afrika}}
>     [[Fil:Map_of_Africa_from_Encyclopaedia_Britannica_1890.jpg|miniatyr|Politisk karta över Afrika från 1890.]] Från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. Holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av Afrika. Portugiserna grundade flera kolonier i västra Afrika och vid nuvarande Moçambique. I mitten av [[1800-talet]] leddes flera expeditioner in i de centrala delarna av Afrika, bland annat av Livingstone och Speke, men också den svenske upptäcktsresanden Andersson. Denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. [[Leopold II av Belgien|Kung Leopolds]] rovdrift på sin privata koloni [[Kongostaten]] är ökänd och kostade miljontals afrikanska liv.
>
>     I Afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket Egypten, republiken Liberia och konungariket Abessinien. Tanger bildade med sitt område en internationell zon. För övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;{{runeberg.org|nfcn|0245.html Supplement Afrika}}&lt;/ref&gt;
>
>     Flertalet kolonier erhöll [[självständighet]] under slutet av 1950-talet och början av 1960-talet. Avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i Rhodesia (nuvarande Zimbabwe), Rwanda, Uganda, Nigeria m.fl. stater.
>
>     == Politik ==
>     {{Fördjupning|Afrikas politik}}
>     [[Fil:Electoral_Democracies_in_Africa,_Freedom_House_2007.png|miniatyr|De blå länderna är demokratier enligt tankesmedjan Freedom House.]]
>     Det postkoloniala Afrikas politik har i hög grad präglats av diktatur, statskupper och brist på [[mänskliga rättigheter]].
>
>     [[Nation]]erna i Afrika har ofta visat sig vara instabila [[stat]]er som efter självständigheten hemsökts av korruption, våldsamma [[statskupp]]er, [[despotism]] och [[dikatur|militärdiktaturer]]. Under perioden [[1960]]-[[1989]] genomfördes över 70 kupper och 13 [[mord]] på [[president]]er. Under det [[kalla kriget]] kämpade [[USA]] och [[Sovjetunionen]] om makten i Afrika, och tillsammans med länder som [[Frankrike]] och [[Kuba]], stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. Enligt en spridd uppfattning beror många [[militär]]a konflikter om [[territorium|territorier]] och [[gräns]]er under Afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter [[Kongokonferensen]], men detta kan inte visas eftersom något jämförelseområde inte finns. Konflikter i Afrika har ofta drivits av tillgången till kontinentens rika [[råvara|råvarutillgångar]] – [[diamant]]er och [[metall]]er som [[guld]], [[platina]], [[uran]] och [[koppar]].
>
>     Korruption och havererade politiska program har lett till utbredd [[undernäring]] och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. Sanitetsproblemen har också orsakat en akut spridning av bland annat [[HIV]]-viruset som leder till [[sjukdom]]en [[AIDS]].
>
>     Trots nöd och umbäranden finns vissa positiva tecken – [[demokrati]]skt valda [[regering]]ar växer i antal även om de inte utgör någon majoritet ännu. Genom påtryckningar från till exempel [[Internationella valutafonden]] (IMF) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. Om man beaktar IMF:s katastrofala misslyckanden i länder som [[Argentina]] och [[Filippinerna]] är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad Afrika faktiskt behöver.
>
>     De afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. Hittills har dock detta inte lyckats särskilt väl. Under [[inbördeskrig]]et i [[Kongo-Kinshasa|Kongo]] var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. Antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     De politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. Organisationen för afrikansk enighet (OAU 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället ''[[Afrikanska unionen]]'', som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. Hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i Darfur-provinsen i Sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. En avgörande skillnad mellan såväl OAU som AU å ena sidan och EU å den andra, är att EU startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. OAU och AU har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. På så sätt liknar AU mer FN än EU.
>
>     == Ekonomi ==
>     {{Fördjupning|Afrikas ekonomi}}
>     [[Fil:Bakweri cocoyam farmer from Cameroon.jpg|250px|miniatyr|Jordbrukare på en [[jams]]odling i Kamerun]]
>     Afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater ([[Juli 2005]]). Afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. På [[FN]]:s Human Development Report från [[2003]] (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i Afrika.
>
>     Det finns dock länder som lyckas bra ekonomiskt, bland andra [[Botswana]], [[Ghana]] och [[Sydafrika]]. [[Etiopien]] har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. Sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat [[guld]] och [[diamanter]] och ett väl fungerande rättsväsen. Det finns i Sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. Det finns även andra afrikanska länder, exempelvis [[Egypten]], som länge haft en väl fungerande handelssektor och ekonomi.
>
>     Kontinentens kust mot [[Atlanten]] återfinns stora fyndigheter av [[Petroleum|olja]]. I [[Nigeria]] finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     Afrikas jordbruk sysselsätter drygt hälften av Afrikas befolkning och är den viktigaste näringen. Cirka 6 % (ungefär 1 814 600&amp;nbsp;km²) av världsdelens yta är uppodlad, mest i savannområdena. I stora delar av de brukningsbara områdena är jordens bördighet ganska låg. De bästa jordarna återfinns i begränsade höglandsområden som i till exempel [[Kamerun]] och [[Östafrika]].
>
>     == Demografi ==
>     {{Fördjupning|Afrikas demografi}}
>     ===Befolkning===
>     Människan, ''[[Homo sapiens|Homo sapiens sapiens]]'', antas härstamma från Afrika och spred sig härifrån till de omgivande kontinenterna. Den svarta befolkningen söder om Sahara tillhör den ursprungliga befolkningen som hela tiden levat i Afrika.
>
>     Norr om Sahara präglar samröret med övriga [[Medelhavet|Medelhavsområdet]] och inte minst sydvästra [[Asien]] både befolkning, språk och kultur.
>
>     På ön [[Madagaskar]], som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från [[Indonesien]]. Under kolonialtiden skedde inflyttning från [[Europa]] och [[Asien]] till södra Afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     Afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga [[födelsetal]] i kombination med sjunkande [[Mortalitet (demografi)|dödstal]] med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. Även om befolkningstätheten varierar kraftigt mellan olika regioner är Afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     Afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. Även här är variationerna stora, och i norra och östra Afrika finns en lång urban tradition.
>
>     === Språk ===
>     {{Fördjupning|Afrikas språk}}
>     [[Fil:African languages SV.png|miniatyr|300px|Språkfamiljer i Afrika]]
>     Uppskattningsvis finns det ungefär 1800 talade [[språk]] i Afrika idag. Vissa av dessa, till exempel [[Swahili]], [[Hausa]], och [[Yoruba (språk)|Yoruba]], talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. Afrika har även många olika [[teckenspråk]]. De flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna [[afroasiatiska språk]], [[nilo-sahariska språk]], [[Niger-Kongospråk]] eller [[khoisanspråk]].
>     Med några få nämnvärda undantag i östra Afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. Till exempel används i ett stort antal afrikanska länder [[franska]] eller [[engelska]] som administrativt språk för stat, handel, utbildning och media. [[Arabiska]], [[portugisiska]], [[afrikaans]] och [[malagassiska]] är andra exempel på språk med ursprung utanför Afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === Religion ===
>     {{Fördjupning|Afrikas religion}}{{Se även|Kristendomen i Afrika}}
>     [[Kristendom]] och [[islam]] är de största religionerna i Afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. Resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. Ett mindre antal afrikaner är [[Hinduism|hinduer]], eller har tro från [[Judendom|judisk]] tradition. Exempel på [[Afrikanska judar|afrikanska folk som tillhör judendomen]] är [[Beta Israel]], [[Lemba]]folk och [[Abayudaya]] i östra [[Uganda]]. Sydafrika och Angola har över 55% kristna katoliker.
>
>     == Kultur ==
>     {{Fördjupning|Afrikas kultur}}
>     Afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. Den mest konventionella distinktionen är den mellan området söder om Sahara och området norr om öknen. De nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. En av de största kulturella identiteterna är den som innefattar alla folk som talar något [[bantuspråk]].
>
>     Det går också att göra en indelning i gamla franska Västafrika å ena sidan, och resten av Afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra Afrika) å andra sidan. En annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. Det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     [[Afrikansk konst]] och [[afrikansk arkitektur]] reflekterar de afrikanska kulturernas diversitet. De äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av [[nassarius]]skal, dessa hittades i [[Blombosgrottan]]. [[Pyramiderna i Giza]] var världens högsta byggnader under 4 000 år. [[Klippkyrkorna i Lalibela]] i [[Etiopien]], bland vilka ''[[Bete Giyorgis]]'' (&quot;St. Göranskyrkan&quot;) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == Regioner och territorier ==
>     [[Fil:Africa-regions.png|miniatyr|Regioner i Afrika:
>     {{legend|#0000ff|[[Nordafrika]]}}
>     {{legend|#00ff00|[[Västafrika]]}}
>     {{legend|#ff00ff|[[Centralafrika]]}}
>     {{legend|#ffa500|[[Östafrika]]}}
>     {{legend|#ff0000|[[Södra Afrika]]}}]]
>     {| class=&quot;wikitable sortable&quot;
>     |+
>     |- bgcolor=&quot;#ECECEC&quot;
>     ! Regioners namn&lt;ref&gt;Enligt [[:Fil:United Nations geographical subregions.png|FN:s regionindelning]] av världen.&lt;br /&gt;&lt;/ref&gt; och&lt;br /&gt; territorier, med flagga
>     ! data-sort-type=number | [[Area]]&lt;br /&gt;(km²)
>     ! data-sort-type=number |[[Folkmängd]]&lt;br /&gt;(2010)&lt;ref name=&quot;esa.un.org1&quot;/&gt;
>     ! data-sort-type=number |[[Befolkningstäthet]]&lt;br /&gt;(per km²)
>     ! [[Huvudstad]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Centralafrika]]''':
>     |-
>     | {{flaggbild2|Angola}}
>     | align=&quot;right&quot; | 1 246 700
>     | align=&quot;right&quot; | 19&amp;nbsp;081&amp;nbsp;912
>     | align=&quot;right&quot; | 15,3
>     | [[Luanda]]
>     |-
>     | {{flaggbild2|Kamerun}}
>     | align=&quot;right&quot; | 475 440
>     | align=&quot;right&quot; | 19 598 889
>     | align=&quot;right&quot; | 41,2
>     | [[Yaoundé]]
>     |-
>     | {{flaggbild2|Centralafrikanska republiken}}
>     | align=&quot;right&quot; | 622 984
>     | align=&quot;right&quot; | 4 401 051
>     | align=&quot;right&quot; | 7,1
>     | [[Bangui]]
>     |-
>     | {{flaggbild2|Tchad}}
>     | align=&quot;right&quot; | 1 284 000
>     | align=&quot;right&quot; | 11 227 208
>     | align=&quot;right&quot; | 8,7
>     | [[N'Djamena]]
>     |-
>     | {{flaggbild2|Kongo-Brazzaville}}
>     | align=&quot;right&quot; | 342 000
>     | align=&quot;right&quot; | 4 042 899
>     | align=&quot;right&quot; | 12,9
>     | [[Brazzaville]]
>     |-
>     | {{flaggbild2|Kongo-Kinshasa}}
>     | align=&quot;right&quot; | 2 345 410
>     | align=&quot;right&quot; | 65 965 795
>     | align=&quot;right&quot; | 28,1
>     | [[Kinshasa]]
>     |-
>     | {{flaggbild2|Ekvatorialguinea}}
>     | align=&quot;right&quot; | 28 051
>     | align=&quot;right&quot; | 700 401
>     | align=&quot;right&quot; | 24,9
>     | [[Malabo]]
>     |-
>     | {{flaggbild2|Gabon}}
>     | align=&quot;right&quot; | 267 667
>     | align=&quot;right&quot; | 1 505 463
>     | align=&quot;right&quot; | 5,6
>     | [[Libreville]]
>     |-
>     | {{flaggbild2|São Tomé och Príncipe}}
>     | align=&quot;right&quot; | 1 001
>     | align=&quot;right&quot; | 165 397
>     | align=&quot;right&quot; | 165,0
>     | [[São Tomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Nordafrika]]''':
>     |-
>     | {{flaggbild2|Algeriet}}
>     | align=&quot;right&quot; | 2 381 740
>     | align=&quot;right&quot; | 35 468 000
>     | align=&quot;right&quot; | 14,9
>     | [[Alger (stad)|Alger]]
>     |-
>     | {{flaggbild2|Egypten}}&lt;ref&gt;[[Egypten]] brukar anses ligga i två världsdelar, Nordafrika (enligt FN:s regionindelning) och Västasien; befolknings- och arealuppgifterna avser endast den afrikanska delen, väster om [[Suezkanalen]].&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 1 001 450
>     | align=&quot;right&quot; | 81 121 000
>     | align=&quot;right&quot; | 81,0
>     | [[Kairo]]
>     |-
>     | {{flaggbild2|Libyen}}
>     | align=&quot;right&quot; | 1 759 540
>     | align=&quot;right&quot; | 6 355 112
>     | align=&quot;right&quot; | 3,6
>     | [[Tripoli]]
>     |-
>     | {{flaggbild2|Marocko}}
>     | align=&quot;right&quot; | 446 550
>     | align=&quot;right&quot; | 31 951 412
>     | align=&quot;right&quot; | 71,5
>     | [[Rabat]]
>     |-
>     | {{flaggbild2|Sudan}}
>     | align=&quot;right&quot; | 1 886 068
>     | align=&quot;right&quot; | 33 551 941
>     | align=&quot;right&quot; | 17,8
>     | [[Khartoum]]
>     |-
>     | {{flaggbild2|Tunisien}}
>     | align=&quot;right&quot; | 163 610
>     | align=&quot;right&quot; | 10 480 934
>     | align=&quot;right&quot; | 64,1
>     | [[Tunis]]
>     |-
>     | {{flaggbild2|Västsahara}} ([[Marocko]])&lt;ref&gt;[[Västsahara]] är ockuperat av [[Marocko]], som också har anspråk på området.&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 266 000
>     | align=&quot;right&quot; | 530 500
>     | align=&quot;right&quot; | 2,0
>     | [[El Aaiún]]
>     |-
>     | colspan=5 | ''Områden som är likställda delar av europeiska stater'':
>     |-
>     | [[Fil:Flag of the Canary Islands.svg|22x20px|border|Kanarieöarna]] [[Kanarieöarna]] ([[Spanien]])&lt;ref&gt;De till [[Spanien]] hörande [[Kanarieöarna]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]] och [[Västsahara]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 7 492
>     | align=&quot;right&quot; | 2 117 519
>     | align=&quot;right&quot; | 282,6
>     | [[Las Palmas de Gran Canaria]],&lt;br /&gt;[[Santa Cruz de Tenerife]]
>     |-
>     | [[Fil:Flag Ceuta.svg|22x20px|border|Ceuta]] [[Ceuta]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Ceuta]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 20
>     | align=&quot;right&quot; | 78 674
>     | align=&quot;right&quot; | 3933,7
>     | –
>     |-
>     | [[Fil:Flag of Madeira.svg|22x20px|Madeira]] [[Madeira]] ([[Portugal]])&lt;ref&gt;Det till [[Portugal]] hörande [[Madeira]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 797
>     | align=&quot;right&quot; | 267 302
>     | align=&quot;right&quot; | 335,4
>     | [[Funchal]]
>     |-
>     | [[Fil:Flag of Melilla.svg|22x20px|Melilla]] [[Melilla]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Melilla]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 12
>     | align=&quot;right&quot; | 73 460
>     | align=&quot;right&quot; | 6121,6
>     | –
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Södra Afrika]]''':
>     |-
>     | {{flaggbild2|Botswana}}
>     | align=&quot;right&quot; | 600 370
>     | align=&quot;right&quot; | 2 006 945
>     | align=&quot;right&quot; | 3,3
>     | [[Gaborone]]
>     |-
>     | {{flaggbild2|Lesotho}}
>     | align=&quot;right&quot; | 30 355
>     | align=&quot;right&quot; | 2 171 318
>     | align=&quot;right&quot; | 71,5
>     | [[Maseru]]
>     |-
>     | {{flaggbild2|Namibia}}
>     | align=&quot;right&quot; | 825 418
>     | align=&quot;right&quot; | 2 283 289
>     | align=&quot;right&quot; | 2,7
>     | [[Windhoek]]
>     |-
>     | {{flaggbild2|Sydafrika}}
>     | align=&quot;right&quot; | 1 219 912
>     | align=&quot;right&quot; | 50 132 817
>     | align=&quot;right&quot; | 41,1
>     | [[Bloemfontein]], [[Kapstaden]], [[Pretoria]] (Tshwane)&lt;ref&gt;[[Bloemfontein]] är [[Sydafrika]]s judiciella huvudstad, medan [[Kapstaden]] är säte för den lagstiftande församlingen och [[Pretoria]] är administrativ huvudstad. Staden Pretoria har inlett en process för att eventuellt byta namn till Tshwane.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Swaziland}}
>     | align=&quot;right&quot; | 17 363
>     | align=&quot;right&quot; | 1 186 056
>     | align=&quot;right&quot; | 68,3
>     | [[Mbabane]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Västafrika]]''':
>     |-
>     | {{flaggbild2|Benin}}
>     | align=&quot;right&quot; | 112 620
>     | align=&quot;right&quot; | 8 849 892
>     | align=&quot;right&quot; | 78,6
>     | [[Porto-Novo]]
>     |-
>     | {{flaggbild2|Burkina Faso}}
>     | align=&quot;right&quot; | 274 200
>     | align=&quot;right&quot; | 16 468 714
>     | align=&quot;right&quot; | 60,1
>     | [[Ouagadougou]]
>     |-
>     | {{flaggbild2|Kap Verde}}
>     | align=&quot;right&quot; | 4 033
>     | align=&quot;right&quot; | 495 999
>     | align=&quot;right&quot; | 123,0
>     | [[Praia]]
>     |-
>     | {{flaggbild2|Elfenbenskusten}}
>     | align=&quot;right&quot; | 322 460
>     | align=&quot;right&quot; | 19 737 800
>     | align=&quot;right&quot; | 61,2
>     | [[Abidjan]], [[Yamoussoukro]]&lt;ref&gt;[[Yamoussoukro]] är [[Elfenbenskusten]]s officiella huvudstad, men [[Abidjan]] fungerar ''[[de facto]]'' som huvudstad.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Gambia}}
>     | align=&quot;right&quot; | 11 300
>     | align=&quot;right&quot; | 1 728 394
>     | align=&quot;right&quot; | 152,9
>     | [[Banjul]]
>     |-
>     | {{flaggbild2|Ghana}}
>     | align=&quot;right&quot; | 239 460
>     | align=&quot;right&quot; | 24 391 823
>     | align=&quot;right&quot; | 101,9
>     | [[Accra]]
>     |-
>     | {{flaggbild2|Guinea}}
>     | align=&quot;right&quot; | 245 857
>     | align=&quot;right&quot; | 9 981 590
>     | align=&quot;right&quot; | 40,6
>     | [[Conakry]]
>     |-
>     | {{flaggbild2|Guinea-Bissau}}
>     | align=&quot;right&quot; | 36 120
>     | align=&quot;right&quot; | 1 515 224
>     | align=&quot;right&quot; | 41,9
>     | [[Bissau]]
>     |-
>     | {{flaggbild2|Liberia}}
>     | align=&quot;right&quot; | 111 370
>     | align=&quot;right&quot; | 3 994 122
>     | align=&quot;right&quot; | 35,9
>     | [[Monrovia]]
>     |-
>     | {{flaggbild2|Mali}}
>     | align=&quot;right&quot; | 1 240 000
>     | align=&quot;right&quot; | 15 369 809
>     | align=&quot;right&quot; | 9,1
>     | [[Bamako]]
>     |-
>     | {{flaggbild2|Mauretanien}}
>     | align=&quot;right&quot; | 1 030 700
>     | align=&quot;right&quot; | 3 459 773
>     | align=&quot;right&quot; | 3,4
>     | [[Nouakchott]]
>     |-
>     | {{flaggbild2|Niger}}
>     | align=&quot;right&quot; | 1 267 000
>     | align=&quot;right&quot; | 15 511 953
>     | align=&quot;right&quot; | 12,2
>     | [[Niamey]]
>     |-
>     | {{flaggbild2|Nigeria}}
>     | align=&quot;right&quot; | 923 768
>     | align=&quot;right&quot; | 158 423 182
>     | align=&quot;right&quot; | 171,5
>     | [[Abuja]]
>     |-
>     | [[Fil:Flag of Saint Helena.svg|22x20px|Sankta Helena]] [[Sankta Helena (ö)|Sankta Helena]] ([[Storbritannien]])
>     | align=&quot;right&quot; | 410
>     | align=&quot;right&quot; | 4 118
>     | align=&quot;right&quot; | 10,0
>     | [[Jamestown, Sankta Helena|Jamestown]]
>     |-
>     | {{flaggbild2|Senegal}}
>     | align=&quot;right&quot; | 196 190
>     | align=&quot;right&quot; | 12 433 728
>     | align=&quot;right&quot; | 63,4
>     | [[Dakar]]
>     |-
>     | {{flaggbild2|Sierra Leone}}
>     | align=&quot;right&quot; | 71 740
>     | align=&quot;right&quot; | 5 867 536
>     | align=&quot;right&quot; | 81,8
>     | [[Freetown]]
>     |-
>     | {{flaggbild2|Togo}}
>     | align=&quot;right&quot; | 56 785
>     | align=&quot;right&quot; | 6 027 798
>     | align=&quot;right&quot; | 106,1
>     | [[Lomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Östafrika]]''':
>     |-
>     | {{flaggbild2|Burundi}}
>     | align=&quot;right&quot; | 27 830
>     | align=&quot;right&quot; | 8 382 849
>     | align=&quot;right&quot; | 301,2
>     | [[Bujumbura]]
>     |-
>     | {{flaggbild2|Komorerna}}
>     | align=&quot;right&quot; | 2 170
>     | align=&quot;right&quot; | 734 750
>     | align=&quot;right&quot; | 338,6
>     | [[Moroni]]
>     |-
>     | {{flaggbild2|Djibouti}}
>     | align=&quot;right&quot; | 23 000
>     | align=&quot;right&quot; | 888 716
>     | align=&quot;right&quot; | 38,6
>     | [[Djibouti (stad)|Djibouti]]
>     |-
>     | {{flaggbild2|Eritrea}}
>     | align=&quot;right&quot; | 121 320
>     | align=&quot;right&quot; | 5 253 676
>     | align=&quot;right&quot; | 43,3
>     | [[Asmara]]
>     |-
>     | {{flaggbild2|Etiopien}}
>     | align=&quot;right&quot; | 1 127 127
>     | align=&quot;right&quot; | 82 949 541
>     | align=&quot;right&quot; | 73,6
>     | [[Addis Abeba]]
>     |-
>     | {{flaggbild2|Kenya}}
>     | align=&quot;right&quot; | 582 650
>     | align=&quot;right&quot; | 40 512 682
>     | align=&quot;right&quot; | 69,5
>     | [[Nairobi]]
>     |-
>     |{{flaggbild2|Madagaskar}}
>     | align=&quot;right&quot; | 587 040
>     | align=&quot;right&quot; | 20 713 819
>     | align=&quot;right&quot; | 35,3
>     | [[Antananarivo]]
>     |-
>     | {{flaggbild2|Malawi}}
>     | align=&quot;right&quot; | 118 480
>     | align=&quot;right&quot; | 14 900 841
>     | align=&quot;right&quot; | 90,3
>     | [[Lilongwe]]
>     |-
>     | {{flaggbild2|Mauritius}}
>     | align=&quot;right&quot; | 2 040
>     | align=&quot;right&quot; | 1 299 172
>     | align=&quot;right&quot; | 636,8
>     | [[Port Louis]]
>     |-
>     | {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>     | align=&quot;right&quot; | 374
>     | align=&quot;right&quot; | 204 114
>     | align=&quot;right&quot; | 545,8
>     | [[Mamoudzou]]
>     |-
>     | {{flaggbild|Moçambique}} [[Moçambique]]
>     | align=&quot;right&quot; | 801 590
>     | align=&quot;right&quot; | 23 390 765
>     | align=&quot;right&quot; | 29,2
>     | [[Maputo]]
>     |-
>     | {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>     | align=&quot;right&quot; | 2 512
>     | align=&quot;right&quot; | 846 068
>     | align=&quot;right&quot; | 336,8
>     | [[Saint-Denis (Réunion)|Saint-Denis]]
>     |-
>     | {{flaggbild2|Rwanda}}
>     | align=&quot;right&quot; | 26 338
>     | align=&quot;right&quot; | 10 624 005
>     | align=&quot;right&quot; | 403,4
>     | [[Kigali]]
>     |-
>     | {{flaggbild2|Seychellerna}}
>     | align=&quot;right&quot; | 455
>     | align=&quot;right&quot; | 86 518
>     | align=&quot;right&quot; | 190,1
>     | [[Victoria, Seychellerna|Victoria]]
>     |-
>     | {{flaggbild2|Somalia}}
>     | align=&quot;right&quot; | 637 657
>     | align=&quot;right&quot; | 9 330 872
>     | align=&quot;right&quot; | 14,6
>     | [[Mogadishu]]
>     |-
>     | {{flaggbild2|Sydsudan}}
>     | align=&quot;right&quot; | 633 783
>     | align=&quot;right&quot; | 8 207 607
>     | align=&quot;right&quot; | 13
>     | [[Juba]]
>     |-
>     | {{flaggbild2|Tanzania}}
>     | align=&quot;right&quot; | 945 087
>     | align=&quot;right&quot; | 44 841 226
>     | align=&quot;right&quot; | 47,4
>     | [[Dodoma]]
>     |-
>     | {{flaggbild2|Uganda}}
>     | align=&quot;right&quot; | 236 040
>     | align=&quot;right&quot; | 33 424 683
>     | align=&quot;right&quot; | 141,6
>     | [[Kampala]]
>     |-
>     | {{flaggbild2|Zambia}}
>     | align=&quot;right&quot; | 752 614
>     | align=&quot;right&quot; | 13 088 570
>     | align=&quot;right&quot; | 17,4
>     | [[Lusaka]]
>     |-
>     | {{flaggbild2|Zimbabwe}}
>     | align=&quot;right&quot; | 390 580
>     | align=&quot;right&quot; | 12 571 454
>     | align=&quot;right&quot; | 32,2
>     | [[Harare]]
>     |- style=&quot; font-weight:bold; &quot;
>     | Total
>     | align=&quot;right&quot; | 30&amp;nbsp;305&amp;nbsp;053
>     | align=&quot;right&quot; | 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;400
>     | align=&quot;right&quot; | 33,7
>     |}
>
>     == Se även ==
>     * [[Nordafrika]]
>     * [[Västafrika]]
>     * [[Östafrika]]
>     * [[Centralafrika]]
>     * [[Södra Afrika]]
>     * [[Afrikas klimat]]
>
>     == Referenser ==
>
>     === Noter ===
>     &lt;references&gt;
>     &lt;ref name=&quot;esa.un.org&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[Förenta nationerna|United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;ref name=&quot;esa.un.org1&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;/references&gt;
>
>     === Källor ===
>     {{enwp|url=https://en.wikipedia.org/w/index.php?title=Africa&amp;oldid=81702686|artikel=Africa}}
>     * [https://www.courses.psu.edu/test/test100_hkr/AFIM/Main_HTML/Intro_1.html Pennsylvania State University – ''African Ethnic Groups'' och andra kartor]
>     * [http://www.pygmies.info/ Pygméerna i Afrika] Kultur och musik av de första urinvånarna i Afrika
>     * Afrika i skolböckerna, M. Palmberg 1987
>
>     == Externa länkar ==
>     {{commonscat|Africa}}
>     {{Wikinews|Afrika}}
>     {{wikiatlas|Africa}}
>
>     {{Afrika}}
>     {{Världens regioner}}
>     {{auktoritetsdata}}
>
>     [[Kategori:Afrika| ]]
>     [[Kategori:Wikipedia:Basartiklar]]</text>
>           <sha1>1ueywtz2sm89v81ujno96unvkkmxpoa</sha1>
>         </revision>
>       </page>), (55447,<page>
>         <title>Amerika</title>
>         <ns>0</ns>
>         <id>3</id>
>         <revision>
>           <id>48343172</id>
>           <parentid>48187017</parentid>
>           <timestamp>2020-10-11T13:36:09Z</timestamp>
>           <contributor>
>             <username>Olympicbandy</username>
>             <id>564328</id>
>           </contributor>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="2728" xml:space="preserve">{{för|Amerika som kortnamn på '''Amerikas förenta stater'''|USA|andra betydelser|Amerika (olika betydelser)}}
>     [[Fil:Americas (orthographic projection).svg|miniatyr|250px|Karta som visar Nord- och Sydamerika, tillsammans benämnda Amerika]]
>     '''Amerika''' (engelska: ''the Americas'') består av landmassorna mellan [[Atlanten]] och [[Stilla havet]] på [[västra halvklotet]], namngivna efter den italienske upptäcktsresanden [[Amerigo Vespucci]]. Amerika uppfattas vanligen idag som två världsdelar, [[Nordamerika]] och [[Sydamerika]].&lt;ref&gt;[https://snl.no/Amerika Store norske leksikon/ Amerika]&lt;/ref&gt; Amerika indelas geografiskt i Nordamerika (med [[Centralamerika]]) och Sydamerika. [[Grönland]] ingår geografiskt i Nordamerika, men räknas politiskt och kulturellt till Europa. Kulturgeografiskt avgränsas också ofta [[Latinamerika]], då inklusive både Central- och ...

In [ ]:
  def sanitize(str:String) = {
    val noDollar = str.replaceAllLiterally("$", "")
    val noBackslash = noDollar.replaceAllLiterally(raw"\", "")
    noBackslash
    }


  

>     sanitize: (str: String)String

In [ ]:
def filterSource(source: String) : String = {
  
  // There is an &amp;nbsp; where the &amp; provides the & in &nbsp;
  // I should therefore replace &amp; first
  
  val noAmp = raw"&amp;".r.replaceAllIn(source, "&")

  // Loop to remove some specific tags
  val tags = Seq("id", "ns", "parentid", "timestamp", "contributor", "comment", "model", "format", "sha1")
  var current = noAmp
  for (tag <- tags) {
    val start = s"<$tag>".r
    val end = s"</$tag>".r
    while (start.findAllIn(current).hasNext) {
      val a = start.findAllIn(current).start
      val b = end.findAllIn(current).start
      val len = current.length
      current = current.substring(0, a) + current.substring(b+tag.length+3, current.length)
      assert (current.length < len)
    }
  }
  val removedSpecificTags = current

  // Loop to remove some specific sections
  val sections = Seq("Se även", "Referenser", "Noter", "Källor", "Externa länkar")
  current = removedSpecificTags
  for (section <- sections) {
    val regexp = s"[=]+ $section [=]+".r
    if (regexp.findAllIn(current).hasNext)
    {
      val a = regexp.findAllIn(current).start
      val pre = current.substring(0, a)
      var post = current.substring(a, current.length)
      post = regexp.replaceAllIn(post, "")
      if ("^=".r.findAllIn(post).hasNext) {
        val b = "^=".r.findAllIn(post).start
        post = post.substring(b, post.length)
      }
      else {
        post = ""        
      }
      current = pre + post
    }
  }

  val lowerCase = current.toLowerCase  
  val noTags = raw"<[^<>]*>".r.replaceAllIn(lowerCase, "")
  
  // A loop to remove innermost curly braces, then outer such things.
  val curly = raw"\{[^\{\}]*\}".r
  current = noTags
  while (curly.findAllIn(current).hasNext)
  {
     current = curly.replaceAllIn(current, "")
  }
  val noCurly = current
  // val noNumbers = raw"\d".r.replaceAllIn(noCurly, "")

  // Unnamed internal links appear in figure captions, confusing later matching
  val noUnnamedInternalLinks = raw"\[\[([^\|\]]*)\]\]".r.replaceAllIn(noCurly, m => m.group(1))

  // Loop to remove internal links and their properties
  current = noUnnamedInternalLinks
  val regexp = raw"\[\[[^\|\[]*\|([^\[\]]*)\]\]".r 
  while (regexp.findAllIn(current).hasNext) {
    val len = current.length
    current = regexp.replaceAllIn(current, m => "[[" + m.group(1) + "]]")
    assert (current.length < len)
  }
  val noNamedInternalLinks = current
  
  val noInternalLinks = raw"\[\[([^\]]*)\]\]".r.replaceAllIn(noNamedInternalLinks, m => m.group(1))
  val noNamedExternalLinks = raw"\[[^ \[]*\ ([^\]]*)\]".r.replaceAllIn(noInternalLinks, m => m.group(1))
  val noExternalLinks = raw"\[([^\]]*)\]".r.replaceAllIn(noNamedExternalLinks, m => m.group(1))
  
  val nbsp = raw"&nbsp;".r.replaceAllIn(noExternalLinks, " ")
  val noQuotes = raw"&quot;".r.replaceAllIn(nbsp, "")
  val noQuoteChars = raw"'".r.replaceAllIn(noQuotes, "")
  // Loop to remove double whitespace characters
  current = noQuoteChars
  val doubleWhitespace = raw"\s(\s)".r
  while (doubleWhitespace.findAllIn(current).hasNext) {
    current = doubleWhitespace.replaceAllIn(current, m => m.group(1))
  }
  val noDoubleWhitespace = current
  noDoubleWhitespace
}
    
    
    


  

>     filterSource: (source: String)String

In [ ]:
def filteringFails(arg: (Long, String)) = {
  try
  {
    filterSource(sanitize(arg._2));
    false 
  }
  catch
  {
    case _ : Throwable => true
  }
}



val example = dump.filter(filteringFails).take(1)

  

>     filteringFails: (arg: (Long, String))Boolean
>     example: Array[(Long, String)] = Array()

In [ ]:
val sane = dump.map({ case (id, source) => (id, sanitize(source))})

  

>     sane: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[29] at map at command-738986526041602:1

In [ ]:
val filtered = sane.map({case (id, source) => (id, filterSource(source))})

  

>     filtered: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[30] at map at command-738986526041604:1

In [ ]:
filtered.take(5)

  

>     res3: Array[(Long, String)] =
>     Array((10671," amager
>     amager är en dansk ö i öresund. öns norra och västra delar tillhör köpenhamn, medan övriga delar upptas av tårnby kommun och dragørs kommun.
>     amager har en yta på 96,29 km² och befolkningen uppgår till 196 047 personer (1/1 2018). på den östra delen av ön finns kastrups flygplats. amager har också flera militära anläggningar (t.ex. kastrup fort och dragör fort), strandparksområdena amager strandpark och kastrup strandpark, flera restauranger i världsklass (t.ex. noma), ett stort naturområde (amager fælled), ett av nordens största köpcentra (fields), och mycket annat. amager förkortas ofta amar för att återspegla det danska uttalet.
>     amager är delvis en konstgjord ö, delvis en naturlig sådan. t.ex. så anlades en konstgjord ö i anslutning till det redan existerande amager strandpark, när denna rustades upp väsentligen 2005. ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området kalvebod fælled (vestamager). ön började uppodlas i större skala under 1500-talets första hälft då kung kristian ii bjöd in nederländska bönder till att odla upp ön för kronans räkning. flertalet slog sig ned i fiskeläget dragør där många än idag har namn med nederländskt ursprung som neels och tønnes.
>     den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i sundbyöster ut mot kusten. under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i vestamager som idag innefattar ett stort naturområde endast några kilometer från köpenhamns centrum. under senare delen av 1900-talet har flygplatsen kastrup som ligger på ön expanderat kraftigt och i samband med byggandet av öresundsförbindelsen började även ett stort område byggas som kallas för ørestad.
>     ön, som är förbunden med övriga köpenhamn genom flera broar, järnväg och sedan 2017 även metro, har under de senaste 10-20 åren förändrats mycket. dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena örestad, islands brygge och amager strand, och dels så har amagers närhet till köpenhamn raderat dess tidigare rykte som en industriell förstad till köpenhamn. idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. amager strand har förändrat bilden av ön. idag är flera delar av ön mycket attraktiva bostadsområden.
>     ön återfick i samband med öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. en motorväg byggdes samtidigt över öns mellersta del. köpenhamns metro går till kastrup och vestamager med ett antal stationer på ön. kring metro- och regionaltågsstationen ørestad har ett stort kontors-, handels- och bostadsområde uppförts under 00-talet. detta sträcker sig från bella center i norr till metrostationen vestamager i söder. även om området inte ligger vid kusten har ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. vid islands brygge finns en äkta sjöstad.
>     före öresundsförbindelsens uppförande hade småstaden dragør en färjeförbindelse med limhamn i södra malmö. på grund av denna fanns i dragør tidigare en betydande gränshandel. denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>     öns västra del, som utvanns från öresund under 1920- och 30-talen, är med undantag för bella center, ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     längst i söder finns den flerhundraåriga skogen kongelunden. på öns östra del, vid öresundskusten, finns amager strandpark med en populär sandstrand. området har omgestaltats, med en konstgjord ö och en lagun innanför. nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>     amager är den tätast befolkade ön i danmark. den näst mest tätbefolkade är thurø.
>     "), (52187," afrika
>     afrika är jordens näst största kontinent (efter eurasien) och även jordens näst största världsdel efter asien, både vad gäller areal och folkmängd. med världsdelens öar inräknade mäter afrika 30 244 050 km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. omkring 22 miljoner km² av dessa ligger i tropikerna vilket gör den afrikanska kontinenten till världens varmaste kontinent. i afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. dess längd i nordlig-sydlig riktning är omkring 8 000 km och dess största bredd omkring 7 800 km.
>     == etymologi ==
>     ordet afrikas etymologiska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika epoker.
>     i äldre tider kallades allt land väster om egypten för libyen. enligt den grekiska historieskrivaren herodotos berättelse seglade feniciska sjömän på uppdrag av den egyptiska kungen neko runt libyen, det vill säga kontinenten afrika redan omkring 600 f.kr.
>     när romarna först använde termen africa terra avsåg de före andra puniska kriget endast trakten kring karthago, det vill säga motsvarande ungefär dagens tunisien där själva staden karthago låg samt stora delar av libyen och algeriet. romarna uppfattade först egypten som en del av asien, och det var först geografen ptolemaios som drog gränsen vid suez. under kejsartiden kom africa att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. namnet ska ha avsett landet där afri (singularis: afer) bodde. afer avsåg namnet på ett folk som kan motsvara dagens berber. vikingarna under 800- och 900-talen kallade norra afrika för blåland.
>     == geografi ==
>     geologiskt är afrika en mycket gammal kontinent, i huvudsak uppbyggd av urberg. under jordens forntid var den en del av jättekontinenten gondwanaland. denna gled isär under juraperioden och kritaperioden för 100-150 miljoner år sedan, och så fick afrika sin nuvarande form.
>     inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050 km² inklusive öarna. afrika åtskiljs från europa av medelhavet och förenas med asien i sitt nordöstra hörn genom näset vid suez, omkring 130 km i bredd. omkring 8 000 km skiljer den nordligaste punkten, ras ben sakka i tunisien, från dess sydligaste, kap agulhas i sydafrika. från kap verde, dess västligaste punkt, till ras hafun i somalia är det omkring 7 400 km. afrikas kuster mäter omkring 28 000 km (1 km kust på 1 057 km²) varav medelhavet står för 5 400 km, atlanten 11 000 km, indiska oceanen 8 700 km och röda havet 2 900 km, och avsaknaden av djupare bukter och betydande halvöar blir uppenbar om man jämför med europas 9 700 000 km² som omgärdas av 32 000 km kust (1 km kust på 278 km² land), eller för det fjordrika norge, där motsvarande siffra 385 000 km² land och en kuststräcka på 20 000 km (1 km kust per 19 km² land).
>     afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. jämfört med de andra kontinenterna utmärker sig afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. landområdena under 180 m ö.h. upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i asien, utgörs av isolerade berg och bergskedjor. generellt ligger de högre platåerna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid röda havets mitt till kongoflodens mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. man kan således dela upp afrika i fyra övergripande landområden:
>     # slättlanden vid kusterna – ofta med mangroveträsk insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # atlasbergen i norr som ligger isolerat av saharaöknen som bitvis ligger under havsnivån.
>     # de södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # de nordliga och västra slättlanden, inklusive sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>     skog täcker cirka en femtedel av afrikas yta.
>     hela afrika har ett tropiskt eller subtropiskt klimat.
>     === fauna ===
>     afrika är huvudsakligen känt för den öppna savannen med väldiga hjordar av hovdjur som afrikansk buffel, gnuer, antiloper, giraffer och zebror. dessa jagas av stora rovdjur som lejon, hyenor och geparder. den afrikanska djungeln befolkas av ormar och primater samt av vattenlevande djur som krokodiler och groddjur. afrika har även det största antalet arter som kan räknas till megafaunan, bland annat afrikanska elefanter, trubbnoshörning och flodhäst, på grund av att kontinenten bara i mindre utsträckning var påverkad av det massutdöende som skedde under pleistocen.
>     == historia ==
>     den första mänskliga migrationen, från kontinenten men även inom afrikas kuster, kan spåras lingvistiskt, kulturellt och genom datoranalyser av genetiskt material. någonstans i östafrika tror dagens forskare att de första hominider uppkom. i etiopien har man hittat ett två miljoner år gammalt skelett som uppkallades lucy, den första kända mänskliga primaten.
>     === tidigare civilisationer och handel ===
>     några av de mest kända afrikanska civilisationerna. civilisationer från olika tidsepoker visas parallellt.lingvistiska spår tyder på att bantufolk emigrerade från västafrika in i khoisanfolkets (de som tidigare kallades bushmän och hottentotter) område och undanträngde dem. bantufolken använde en uppsättning grödor, bland annat kassava och jams, som var mer lämpade för jordbruk i det tropiska afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. bantufolkens traditionella bosättningsområde sträcker sig från saharaöknen till de tempererade regionerna i söder. historiskt har deras vapen varit spjut, sköld, pil och båge. det för khoisanspråkens karaktäristiska klickljuden återfinns i nguni- och bantuspråk som till exempel xhosa och zulu. khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra afrika i länderna kring kalahariöknen.
>     afrika söder om sahara har bestått olika stater och riken, mindre stadsstater samt stammar och klaner. det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska civilisationerna. det fanns även statsbildningar i nubien, ghana, timbuktu, kanem-bornu och i nuvarande zimbabwe.
>     nordafrika kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds islam och den arabiska kulturen genom sahara och vidare söderut. arabiska och persiska handelsmän spred likaså islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till java, därmed den etniska, kulturella och språkliga mångfalden på östafrikas kust, madagaskar och komorerna.
>     === arabvärlden och europa utforskar kontinenten ===
>     det arabiska intresset för afrika går långt tillbaka, men handel med slavar från centra som lamu och zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.kr. de miljoner slavar som exporterades till arabvärlden och österut under den arabiska slavhandeln finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel sudan.&lt;ref&gt;human rights watch bbc world&lt;/ref&gt;
>     under 1300-talet började vissa europeiska handelsexpeditioner få kontakt med nord- och västafrika.
>     från 1500-talets mitt fram till förbudet mot slavhandel i storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst väst- och centralafrika. flera miljoner afrikaner såldes på detta vis som slavar till främst de europeiska kolonierna i amerika.
>     tidiga kolonisatörer var araber, portugiser och spanjorer. därefter koloniserades kontinenten av framförallt av storbritannien, frankrike, spanien, italien, belgien, nederländerna, tyskland och portugal.
>     === koloniseringen ===
>     politisk karta över afrika från 1890. från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av afrika. portugiserna grundade flera kolonier i västra afrika och vid nuvarande moçambique. i mitten av 1800-talet leddes flera expeditioner in i de centrala delarna av afrika, bland annat av livingstone och speke, men också den svenske upptäcktsresanden andersson. denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. kung leopolds rovdrift på sin privata koloni kongostaten är ökänd och kostade miljontals afrikanska liv.
>     i afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket egypten, republiken liberia och konungariket abessinien. tanger bildade med sitt område en internationell zon. för övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;&lt;/ref&gt;
>     flertalet kolonier erhöll självständighet under slutet av 1950-talet och början av 1960-talet. avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i rhodesia (nuvarande zimbabwe), rwanda, uganda, nigeria m.fl. stater.
>     == politik ==
>     de blå länderna är demokratier enligt tankesmedjan freedom house.
>     det postkoloniala afrikas politik har i hög grad präglats av diktatur, statskupper och brist på mänskliga rättigheter.
>     nationerna i afrika har ofta visat sig vara instabila stater som efter självständigheten hemsökts av korruption, våldsamma statskupper, despotism och militärdiktaturer. under perioden 1960-1989 genomfördes över 70 kupper och 13 mord på presidenter. under det kalla kriget kämpade usa och sovjetunionen om makten i afrika, och tillsammans med länder som frankrike och kuba, stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. enligt en spridd uppfattning beror många militära konflikter om territorier och gränser under afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter kongokonferensen, men detta kan inte visas eftersom något jämförelseområde inte finns. konflikter i afrika har ofta drivits av tillgången till kontinentens rika råvarutillgångar – diamanter och metaller som guld, platina, uran och koppar.
>     korruption och havererade politiska program har lett till utbredd undernäring och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. sanitetsproblemen har också orsakat en akut spridning av bland annat hiv-viruset som leder till sjukdomen aids.
>     trots nöd och umbäranden finns vissa positiva tecken – demokratiskt valda regeringar växer i antal även om de inte utgör någon majoritet ännu. genom påtryckningar från till exempel internationella valutafonden (imf) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. om man beaktar imf:s katastrofala misslyckanden i länder som argentina och filippinerna är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad afrika faktiskt behöver.
>     de afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. hittills har dock detta inte lyckats särskilt väl. under inbördeskriget i kongo var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>     de politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. organisationen för afrikansk enighet (oau 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället afrikanska unionen, som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i darfur-provinsen i sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. en avgörande skillnad mellan såväl oau som au å ena sidan och eu å den andra, är att eu startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. oau och au har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. på så sätt liknar au mer fn än eu.
>     == ekonomi ==
>     jordbrukare på en jamsodling i kamerun
>     afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater (juli 2005). afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. på fn:s human development report från 2003 (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i afrika.
>     det finns dock länder som lyckas bra ekonomiskt, bland andra botswana, ghana och sydafrika. etiopien har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat guld och diamanter och ett väl fungerande rättsväsen. det finns i sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. det finns även andra afrikanska länder, exempelvis egypten, som länge haft en väl fungerande handelssektor och ekonomi.
>     kontinentens kust mot atlanten återfinns stora fyndigheter av olja. i nigeria finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>     afrikas jordbruk sysselsätter drygt hälften av afrikas befolkning och är den viktigaste näringen. cirka 6 % (ungefär 1 814 600 km²) av världsdelens yta är uppodlad, mest i savannområdena. i stora delar av de brukningsbara områdena är jordens bördighet ganska låg. de bästa jordarna återfinns i begränsade höglandsområden som i till exempel kamerun och östafrika.
>     == demografi ==
>     ===befolkning===
>     människan, homo sapiens sapiens, antas härstamma från afrika och spred sig härifrån till de omgivande kontinenterna. den svarta befolkningen söder om sahara tillhör den ursprungliga befolkningen som hela tiden levat i afrika.
>     norr om sahara präglar samröret med övriga medelhavsområdet och inte minst sydvästra asien både befolkning, språk och kultur.
>     på ön madagaskar, som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från indonesien. under kolonialtiden skedde inflyttning från europa och asien till södra afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>     afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga födelsetal i kombination med sjunkande dödstal med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. även om befolkningstätheten varierar kraftigt mellan olika regioner är afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>     afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. även här är variationerna stora, och i norra och östra afrika finns en lång urban tradition.
>     === språk ===
>     språkfamiljer i afrika
>     uppskattningsvis finns det ungefär 1800 talade språk i afrika idag. vissa av dessa, till exempel swahili, hausa, och yoruba, talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. afrika har även många olika teckenspråk. de flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna afroasiatiska språk, nilo-sahariska språk, niger-kongospråk eller khoisanspråk.
>     med några få nämnvärda undantag i östra afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. till exempel används i ett stort antal afrikanska länder franska eller engelska som administrativt språk för stat, handel, utbildning och media. arabiska, portugisiska, afrikaans och malagassiska är andra exempel på språk med ursprung utanför afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>     === religion ===
>     kristendom och islam är de största religionerna i afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. ett mindre antal afrikaner är hinduer, eller har tro från judisk tradition. exempel på afrikanska folk som tillhör judendomen är beta israel, lembafolk och abayudaya i östra uganda. sydafrika och angola har över 55% kristna katoliker.
>     == kultur ==
>     afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. den mest konventionella distinktionen är den mellan området söder om sahara och området norr om öknen. de nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. en av de största kulturella identiteterna är den som innefattar alla folk som talar något bantuspråk.
>     det går också att göra en indelning i gamla franska västafrika å ena sidan, och resten av afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra afrika) å andra sidan. en annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>     afrikansk konst och afrikansk arkitektur reflekterar de afrikanska kulturernas diversitet. de äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av nassariusskal, dessa hittades i blombosgrottan. pyramiderna i giza var världens högsta byggnader under 4 000 år. klippkyrkorna i lalibela i etiopien, bland vilka bete giyorgis (st. göranskyrkan) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>     == regioner och territorier ==
>     regioner i afrika:
>     "), (55447," amerika
>     karta som visar nord- och sydamerika, tillsammans benämnda amerika
>     amerika (engelska: the americas) består av landmassorna mellan atlanten och stilla havet på västra halvklotet, namngivna efter den italienske upptäcktsresanden amerigo vespucci. amerika uppfattas vanligen idag som två världsdelar, nordamerika och sydamerika.&lt;ref&gt;store norske leksikon/ amerika&lt;/ref&gt; amerika indelas geografiskt i nordamerika (med centralamerika) och sydamerika. grönland ingår geografiskt i nordamerika, men räknas politiskt och kulturellt till europa. kulturgeografiskt avgränsas också ofta latinamerika, då inklusive både central- och sydamerika samt mexiko, och angloamerika, som är usa och kanada. områdets totala landyta är ungefär 42 miljoner kvadratkilometer och är således något mindre än asiens.
>     ofta används amerika och amerikansk för enbart landet usa, amerikas förenta stater, vilket medför en förväxlingsrisk med dubbelkontinenten som helhet men är jämförbart med hur andra landsnamn förkortas i dagligt tal (exempelvis mexiko, vars fullständiga namn översatt till svenska är mexikos förenta stater). på engelska, portugisiska och spanska kommer man ofta runt problemet genom att de två världsdelarna kallas the americas respektive las américas (två amerikorna, ett uttryck som dock aldrig har etablerats på svenska).
>     == namnet ==
>     amerika namngavs efter amerigo vespucci, som var den första som insåg att amerika var en kontinent som nyligen kommit till kännedom för européer, som inte var östindien. namnet används 1507 för första gången i en världskarta som skapades av den tyske geografen martin waldseemüller och beskriver där sydamerika. 1511 omnämnes det i ett engelskt skådespel och 1516 används det av leonardo da vinci på hans världskarta och vann alltså snabb utbredning. spanjorerna brukade i början ha andra namn. de kallade landet för den nya världen eller las indias occidentales (västra indien). för sydamerika behöll de under hela 1500-talet namnen peruana och brasilia. först på 1600-talet blev namnet amerika allmänt antaget.
>     == historia ==
>     * se amerikas historia
>     "), (58373," abba (olika betydelser)
>     abba kan avse:
>     * abba – en tidigare svensk popgrupp
>     ** abba (musikalbum) – ett album av gruppen
>     * abba seafood – ett fiskkonservföretag
>     * abba (teologi) – ett arameiskt ord
>     * abba (hederstitel) – en hederstitel och förtroligt tilltal
>     ==geografi==
>     &lt;!--notera: om platser läggs till, tas bort eller ordningen på platserna ändras, bör också mallen kartposition under samma rubrik korrigeras för att kartan ska förbli rättvisande.--&gt;
>     === australien ===
>     * abba river
>     === centralafrikanska republiken ===
>     # abba (subprefektur), préfecture de la nana-mambéré, &lt;!--geonames id 9143482--&gt;
>     === etiopien ===
>     # abba (periodiskt vattendrag), amhara, &lt;!--geonames id 345835--&gt;
>     # ābaya hāyk, sjö, &lt;!--geonames id 345844--&gt;
>     # abba (skog), oromia, &lt;!--geonames id 6642287--&gt;
>     kategori:robotskapade förgreningar 2016-01
>     kategori:robotskapade centralafrikanska republikenförgreningar
>     kategori:robotskapade etiopienförgreningar "), (64397," abbekås &lt;!-- slut på mall:ortsfakta sverige --&gt;
>     abbekås (uttalas abbekå-s)&lt;ref&gt;runeberg.org: svenska ortnamn med uttalsuppgifter sida 4&lt;/ref&gt; är en tätort i skivarps distrikt (skivarps socken) på söderslätt i skurups kommun.
>     == historia ==
>     abbekås var från början ett fiskeläge på skånes sydkust mellan ystad och trelleborg, omtalad redan på medeltiden. onsdagen den 13 november 1872 förstördes stora delar av byn av en stormflod kallad backafloden, framkallad av en orkan som drog in över sydkusten från nordost. 34 hus förstördes, alla fiskeredskap, båtar och befolkningens vinterförråd. vattenståndet ökade till 3,6 meter över det normala, vilket fortfarande är svenskt rekord. redan året därpå inleddes med hjälp av statsbidrag arbetet med en riktig hamn, och fisket och kusthandeln kunde åter komma igång. en ny storhetstid för fisket inträffade under andra världskriget.
>     2011 renoverades pirar och kajer, och inloppet gjordes om för att öka säkerheten i yttre hamnbassängen.
>     === befolkningsutveckling ===
>     ==namnet==
>     namnet skrevs 1536 abbekassz. efterleden är kås, båtplats, mindre hamn. förleden innehåller förmodligen mansnamnet abbi.&lt;ref&gt;&lt;/ref&gt;.
>     ==samhället==
>     abbekås hamn består av en yttre och en inre hamnbassäng, där den inre används av fiskarna och den yttre är till för fritidsseglare och gästbåtar. här finns såväl hamnkrog och hotell som ett modernt tvätt/dusch/toaletthus för besökare. längre upp i byn finns ytterligare en restaurang och en livsmedelsaffär, ett konstgalleri och två bed&breakfast-ställen. strax öster om byn ligger abbekås golfklubb.
>     ==abbekås i musiken==
>     abbekås är omsjunget av edvard persson i sången abbekåsagåsen joakim.
>     == kända profiler ==
>     *lorens brolin, spelman
>     == bilder ==
>     &lt;gallery&gt;
>     abbekås 2.jpg|abbekås hamn
>     hamnen abbekås.jpg|abbekås hamn
>     magasinet abbekås.jpg|magasinet vid abbekås hamn
>     coast line abbekås.jpg|strandlinjen väster om abbekås hamn
>     abbekås 3.jpg|abbekåsagåsen joakim
>     &lt;/gallery&gt;
>     ==omgivande orter==
>     "))